In [1]:
import re
import numpy as np
import pandas as pd
import requests
from lxml import etree
from tqdm.notebook import tqdm

# 1.抓最新匯率

In [2]:
def get_money_now():
    response = requests.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
    html = etree.HTML(response.text)
    money_type = [i.strip() for i in html.xpath('//tbody//td[@data-table="幣別"]//div[@class="visible-phone print_hide"]//text()')]
    buy = html.xpath('//tbody//td[@data-table="本行現金買入" and @class="rate-content-cash text-right print_hide"]//text()')
    sell = html.xpath('//tbody//td[@data-table="本行現金賣出" and @class="rate-content-cash text-right print_hide"]//text()')
    df = pd.DataFrame({'幣別':money_type,'本行現金買入':buy,'本行現金賣出':sell})
    return df

In [3]:
today = get_money_now()
today

,幣別,本行現金買入,本行現金賣出
0,美金 (USD),29.745,30.415
1,港幣 (HKD),3.714,3.918
2,英鎊 (GBP),38.88,41
3,澳幣 (AUD),20.32,21.1
4,加拿大幣 (CAD),22.4,23.31
5,新加坡幣 (SGD),21.68,22.59
6,瑞士法郎 (CHF),29.86,31.06
7,日圓 (JPY),0.2659,0.2787
8,南非幣 (ZAR),-,-
9,瑞典幣 (SEK),2.83,3.35


# 2.抓各國所有月份的匯率

In [4]:
county_list = today['幣別'].apply(lambda x : re.findall(r'\((.+?)\)',x)[0]).values
month_list = ['0'+i for i in np.arange(1,13).astype('str') if len(i)== 1] + [i for i in np.arange(1,13).astype('str') if len(i)== 2]
year_list = ['2018','2019']

In [5]:
def get_month_price(country,year,month):
    response = requests.get('https://rate.bot.com.tw/xrt/quote/{}-{}/{}'.format(year,month,country))
    html = etree.HTML(response.text)
    price_both = np.array(html.xpath('//td[@class="rate-content-cash text-right print_table-cell"]//text()')).reshape(-1,2)
    date = html.xpath('//td[@class="text-center"]//text()')
    buy = price_both[:,0]
    sell = price_both[:,1]
    df = pd.DataFrame({'掛牌日期':date,'本行買入':buy,'本行賣出':sell})
    df['幣別'] = country
    return df

In [6]:
sum_table = pd.DataFrame({'掛牌日期':[],'本行買入':[],'本行賣出':[],'幣別':[]})
for i in tqdm(county_list):
    for j in year_list:
        for k in month_list:
            df = get_month_price(i,j,k)
            sum_table = pd.concat([sum_table,df],axis=0,ignore_index=True)

In [7]:
sum_table

,掛牌日期,本行買入,本行賣出,幣別
0,2018/01/31,28.78,29.322,USD
1,2018/01/30,28.87,29.412,USD
2,2018/01/29,28.795,29.337,USD
3,2018/01/26,28.735,29.277,USD
4,2018/01/25,28.695,29.237,USD
5,2018/01/24,28.86,29.402,USD
6,2018/01/23,28.91,29.452,USD
7,2018/01/22,28.99,29.532,USD
8,2018/01/19,29.06,29.602,USD
9,2018/01/18,29.2,29.742,USD
